In [1]:
import sys
sys.path.append('../')

In [15]:
import torch
from models.unet import UNet

In [3]:
model = UNet()

In [10]:
all_layers = dict([*model.named_modules()])
intermediate_layers = []
for name, layer in all_layers.items():
    if '.decoder.blocks' in name:
        intermediate_layers.append(name)

In [16]:
test_input = torch.ones(1, 3, 256, 256)

In [18]:
res_encoder = model.encoder(test_input)

In [19]:
for res in res_encoder:
    print(res.shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 64, 128, 128])
torch.Size([1, 256, 64, 64])
torch.Size([1, 512, 32, 32])
torch.Size([1, 1024, 16, 16])
torch.Size([1, 2048, 8, 8])


In [20]:
features = res_encoder

In [21]:
decoder_res = []

In [26]:
model.encoder.out_channels

(3, 64, 256, 512, 1024, 2048)

In [22]:
features = features[1:]  # remove first skip with same spatial resolution
features = features[::-1]  # reverse channels to start from head of encoder

head = features[0]
skips = features[1:]

x = model.decoder.center(head)
for i, decoder_block in enumerate(model.decoder.blocks):
    skip = skips[i] if i < len(skips) else None
    x = decoder_block(x, skip)
    decoder_res.append(x)

In [23]:
for res in decoder_res:
    print(res.shape)

torch.Size([1, 256, 16, 16])
torch.Size([1, 128, 32, 32])
torch.Size([1, 64, 64, 64])
torch.Size([1, 32, 128, 128])
torch.Size([1, 16, 256, 256])


In [11]:
intermediate_layers

[]

In [27]:
%load_ext autoreload
%autoreload 2

In [34]:
from models.unet_le import UNetLE

In [35]:
model = UNetLE()

In [42]:
res = model({'image':test_input})

In [40]:
for res in res:
    print(res.shape)

torch.Size([1, 2, 128, 128])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 32, 32])
torch.Size([1, 2, 16, 16])
torch.Size([1, 2, 8, 8])
torch.Size([1, 2, 16, 16])
torch.Size([1, 2, 32, 32])
torch.Size([1, 2, 64, 64])
torch.Size([1, 2, 128, 128])
torch.Size([1, 2, 256, 256])


In [45]:
torch.stack([test_input, test_input]).shape

torch.Size([2, 1, 3, 256, 256])